In [1]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML
CLIENT_ID = 'TJ0D0H1JMZCX3N1YYXGL0D3VSNIOHFO1DSNFW3ZIWIPB4Q2N' # your Foursquare ID
CLIENT_SECRET = 'ODLIF4YATNNGVV0FMCPA1KEIKR1GZUPZGYRPWSENJ5DNHWIX' # your Foursquare Secret
VERSION = '20190530'

# Battle of the Neighborhoods - Capstone Project Week 2

## **1. Introduction/Business Problem**

Large cities can provide businesses the opportunity to grow, expand, and become more well known. Houston, Texas is the 4th largest city in the United States and rapidly growing, making it a great city for a new business to access a large group of new potential clients. The famous saying in real estate, "Location, location, location" is applicable to the business world as well. Location has the potential to be the deciding factor in whether a business succeeds or fails. For simplicity, different suburban areas in the West Houston will be the focal point of this project. Factors that will be considered include
- Which suburban areas have the fewest number restaurants, thereby minimizing potential competition
- Other venues in the suburban area that might attract restaurant patrons (movie theaters, shopping centers, etc.)

Potential people who could utilize the results of this project are entrepreneurs interested in opening up a restaurant. However, the target audience would be the investors or lending institution. Showing them that research has been done to make a data driven decision on the location of the future business shows the investors that decisions aren't made on a whim and that logical thought has been put into the decision in such a way that the business would be most profitable.

## **2. Data**

As Houston is a very large city, it also has a diverse amount of suburban areas. In this project, we will focus on the folowing suburban areas of Houston


1. Memorial City
2. Katy
3. Cypress
4. Jersey Village
5. Sugar Land

## Creating DataFrame

In [2]:
#Coordinates of Suburban Areas in Houston
memorial_latitude = 29.7838
memorial_longitude = -95.5538
katy_latitude = 29.7858
katy_longitude = -95.8244
cypress_latitude = 29.9717
cypress_longitude = -95.6938
jv_latitude = 29.8877
jv_longitude = -95.5630
sl_latitude = 29.6197
sl_longitude = -95.6349

#Dataframe containing suburban areas and coordinates
Data = {'Area': ['Memorial City', 'Katy', 'Cypress', 'Jersey Village', 'Sugar Land'],
        'Latitude': [memorial_latitude, katy_latitude, cypress_latitude, jv_latitude, sl_latitude],
        'Longitude': [memorial_longitude, katy_longitude, cypress_longitude, jv_longitude, sl_longitude]}

df = pd.DataFrame(Data, columns = ['Area', 'Latitude','Longitude'])
df

,Area,Latitude,Longitude
0,Memorial City,29.7838,-95.5538
1,Katy,29.7858,-95.8244
2,Cypress,29.9717,-95.6938
3,Jersey Village,29.8877,-95.5630
4,Sugar Land,29.6197,-95.6349


## Creating a Map of Houston, Labeling Suburban Areas of Interest

In [3]:
map_houston = folium.Map(location=[29.7604, -95.3698], zoom_start=10)

for Area, Latitude, Longitude in zip(df['Area'], df['Latitude'], df['Longitude']):
    label = '{}'.format(Area)
    label = folium.Popup(label)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='coral',
        fill_opacity=0.7,
        parse_html=False).add_to(map_houston)
    
map_houston

In [4]:
#Function written to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Memorial City

In [5]:
memorial_limit = 100
memorial_radius = 3500

#Defining URL's for Foursquare API Requests
memorial_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, memorial_latitude, memorial_longitude, VERSION, memorial_radius, memorial_limit)

#Examining results
memorial_results = requests.get(memorial_url).json()

memorial_venues = memorial_results['response']['groups'][0]['items']
nearby_memorial_venues = json_normalize(memorial_venues)
filtered_memorial_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_memorial_venues =nearby_memorial_venues.loc[:, filtered_memorial_columns]
#     filter the category for each row
nearby_memorial_venues['venue.categories'] = nearby_memorial_venues.apply(get_category_type, axis=1)
#     clean columns
nearby_memorial_venues.columns = [col.split(".")[-1] for col in nearby_memorial_venues.columns]

In [6]:
#Finding Restaurants in Memorial City
memorial_restaurants = nearby_memorial_venues[nearby_memorial_venues['categories'].str.contains('Restaurant')]
memorial_restaurants.reset_index(drop = True, inplace = True)
print ('There are', len(memorial_restaurants),'restaurants in Memorial City')
memorial_restaurants.head()

There are 38 restaurants in Memorial City


,name,categories,lat,lng
0,Fadi's Mediterranean Grill,Mediterranean Restaurant,29.782859,-95.554195
1,Pappadeaux Seafood Kitchen,Seafood Restaurant,29.783318,-95.556251
2,Pappasito's Cantina,Mexican Restaurant,29.783314,-95.555078
3,Sal y Pimienta Kitchen,South American Restaurant,29.781718,-95.560532
4,Eddie V's Prime Seafood,Seafood Restaurant,29.778897,-95.560740


In [7]:
#Finding other venues people might eat at after visiting in Memorial City
memorial_stores = nearby_memorial_venues[nearby_memorial_venues['categories'].str.contains('Store')]
memorial_stores.reset_index(drop = True, inplace = True)
print ('There are', len(memorial_stores),'stores in Memorial City')

memorial_shops = nearby_memorial_venues[nearby_memorial_venues['categories'].str.contains('Shop')]
memorial_shops.reset_index(drop = True, inplace = True)
print ('There are', len(memorial_shops),'shops and malls in Memorial City')

memorial_beauty = nearby_memorial_venues[nearby_memorial_venues['categories'].str.contains('Beauty')]
memorial_beauty.reset_index(drop = True, inplace = True)
print ('There are', len(memorial_beauty),'beauty services in Memorial City')

memorial_theater = nearby_memorial_venues[nearby_memorial_venues['categories'].str.contains('Theater')]
memorial_theater.reset_index(drop = True, inplace = True)
print ('There are', len(memorial_theater),'theaters in Memorial City')

memorial_hotel = nearby_memorial_venues[nearby_memorial_venues['categories'].str.contains('Hotel')]
memorial_hotel.reset_index(drop = True, inplace = True)
print ('There are', len(memorial_hotel),'hotels in Memorial City')

There are 12 stores in Memorial City
There are 17 shops and malls in Memorial City
There are 2 beauty services in Memorial City
There are 2 theaters in Memorial City
There are 3 hotels in Memorial City


In [8]:
memorial_frames = [memorial_stores, memorial_shops, memorial_beauty, memorial_theater, memorial_hotel]

memorial_other_venues = pd.concat(memorial_frames)
memorial_other_venues.reset_index(drop = True, inplace = True)

print ('Considering stores, shops, beauty services, theaters, and hotels, there are', len(memorial_other_venues),'other venues in Memorial City people might visit before eating')
memorial_other_venues.head()

Considering stores, shops, beauty services, theaters, and hotels, there are 36 other venues in Memorial City people might visit before eating


,name,categories,lat,lng
0,Kendra Scott,Jewelry Store,29.780714,-95.561199
1,"Spec's Wines, Spirits & Finer Foods",Liquor Store,29.777825,-95.563456
2,Bed Bath & Beyond,Furniture / Home Store,29.782750,-95.557524
3,Macy's,Department Store,29.781000,-95.540000
4,West Elm,Furniture / Home Store,29.778849,-95.560726


In [9]:
#Mapping all venues in Memorial City
map_memorial = folium.Map(location=[memorial_latitude, memorial_longitude], zoom_start=14)

#Mapping restaurants in Memorial City
for name, lat, lng in zip(memorial_restaurants['name'], memorial_restaurants['lat'], memorial_restaurants['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(map_memorial)
    
#Mapping stores and shops in Memorial City
for name, lat, lng in zip(memorial_stores['name'], memorial_stores['lat'], memorial_stores['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_memorial)
for name, lat, lng in zip(memorial_shops['name'], memorial_shops['lat'], memorial_shops['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_memorial)
    
#Mapping beauty services in Memorial City
for name, lat, lng in zip(memorial_beauty['name'], memorial_beauty['lat'], memorial_beauty['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7,
        parse_html=False).add_to(map_memorial)
    
#Mapping theaters in Memorial City
for name, lat, lng in zip(memorial_theater['name'], memorial_theater['lat'], memorial_theater['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_memorial)
    
#Mapping hotels in Memorial City
for name, lat, lng in zip(memorial_hotel['name'], memorial_hotel['lat'], memorial_hotel['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_memorial)
    
map_memorial

# Katy

In [10]:
katy_limit = 100
katy_radius = 5700
#Defining URL's for Foursquare API Requests
katy_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, katy_latitude, katy_longitude, VERSION, katy_radius, katy_limit)

#Examining results
katy_results = requests.get(katy_url).json()


katy_venues = katy_results['response']['groups'][0]['items']
nearby_katy_venues = json_normalize(katy_venues)
filtered_katy_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_katy_venues =nearby_katy_venues.loc[:, filtered_katy_columns]
#     filter the category for each row
nearby_katy_venues['venue.categories'] = nearby_katy_venues.apply(get_category_type, axis=1)
#     clean columns
nearby_katy_venues.columns = [col.split(".")[-1] for col in nearby_katy_venues.columns]       

In [11]:
#Finding other venues people might eat at after visiting in Katy
katy_stores = nearby_katy_venues[nearby_katy_venues['categories'].str.contains('Store')]
katy_stores.reset_index(drop = True, inplace = True)
print ('There are', len(katy_stores),'stores in Katy')

katy_shops = nearby_katy_venues[nearby_katy_venues['categories'].str.contains('Shop')]
katy_shops.reset_index(drop = True, inplace = True)
print ('There are', len(katy_shops),'shops and malls in Katy')

katy_beauty = nearby_katy_venues[nearby_katy_venues['categories'].str.contains('Beauty')]
katy_beauty.reset_index(drop = True, inplace = True)
print ('There are', len(katy_beauty),'beauty services in Katy')

katy_theater = nearby_katy_venues[nearby_katy_venues['categories'].str.contains('Theater')]
katy_theater.reset_index(drop = True, inplace = True)
print ('There are', len(katy_theater),'theaters in Katy')

katy_hotel = nearby_katy_venues[nearby_katy_venues['categories'].str.contains('Hotel')]
katy_hotel.reset_index(drop = True, inplace = True)
print ('There are', len(katy_hotel),'hotels in Katy')

There are 25 stores in Katy
There are 15 shops and malls in Katy
There are 0 beauty services in Katy
There are 1 theaters in Katy
There are 2 hotels in Katy


In [12]:
#Finding Restaurants in Katy
katy_restaurants = nearby_katy_venues[nearby_katy_venues['categories'].str.contains('Restaurant')]
katy_restaurants.reset_index(drop = True, inplace = True)
print ('There are', len(katy_restaurants),'restaurants in Katy')
katy_restaurants.head()

There are 27 restaurants in Katy


,name,categories,lat,lng
0,Los Cucos Mexican Cafe,Mexican Restaurant,29.785589,-95.824959
1,Snappy's Cafe & Grill,American Restaurant,29.785492,-95.824028
2,Texas Tradition,American Restaurant,29.785474,-95.812354
3,Chick-fil-A,Fast Food Restaurant,29.774120,-95.819284
4,Willy BURGER,American Restaurant,29.785559,-95.829832


In [13]:
katy_frames = [katy_stores, katy_shops, katy_beauty, katy_theater, katy_hotel]

katy_other_venues = pd.concat(katy_frames)
katy_other_venues.reset_index(drop = True, inplace = True)

print ('Considering stores, shops, beauty services, theaters, and hotels, there are', len(katy_other_venues),'other venues in Katy people might visit before eating')
katy_other_venues.head()

Considering stores, shops, beauty services, theaters, and hotels, there are 43 other venues in Katy people might visit before eating


,name,categories,lat,lng
0,H-E-B,Grocery Store,29.773434,-95.822434
1,Polo Ralph Lauren Factory Store,Clothing Store,29.775868,-95.810094
2,H&M,Clothing Store,29.775677,-95.808875
3,Coach Factory Outlet,Accessories Store,29.777602,-95.809983
4,Banana Republic,Clothing Store,29.775970,-95.809848


In [14]:
#Mapping all venues in Katy
map_katy = folium.Map(location=[katy_latitude, katy_longitude], zoom_start=13)

#Mapping restaurants in Katy
for name, lat, lng in zip(katy_restaurants['name'], katy_restaurants['lat'], katy_restaurants['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(map_katy)
    
#Mapping stores and shops in Katy
for name, lat, lng in zip(katy_stores['name'], katy_stores['lat'], katy_stores['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_katy)
for name, lat, lng in zip(katy_shops['name'], katy_shops['lat'], katy_shops['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_katy)
    
#Mapping beauty services in Katy
for name, lat, lng in zip(katy_beauty['name'], katy_beauty['lat'], katy_beauty['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7,
        parse_html=False).add_to(map_katy)
    
#Mapping theaters in Katy
for name, lat, lng in zip(katy_theater['name'], katy_theater['lat'], katy_theater['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_katy)
    
#Mapping hotels in Katy
for name, lat, lng in zip(katy_hotel['name'], katy_hotel['lat'], katy_hotel['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_katy)
    
map_katy

# Cypress

In [15]:
cypress_limit = 100
cypress_radius = 4681

#Defining URL's for Foursquare API Requests
cypress_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, cypress_latitude, cypress_longitude, VERSION, cypress_radius, cypress_limit)

#Examining results
cypress_results = requests.get(cypress_url).json()

cypress_venues = cypress_results['response']['groups'][0]['items']
nearby_cypress_venues = json_normalize(cypress_venues)
filtered_cypress_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_cypress_venues =nearby_cypress_venues.loc[:, filtered_cypress_columns]
#     filter the category for each row
nearby_cypress_venues['venue.categories'] = nearby_cypress_venues.apply(get_category_type, axis=1)
#     clean columns
nearby_cypress_venues.columns = [col.split(".")[-1] for col in nearby_cypress_venues.columns]

In [16]:
#Finding Restaurants in Cypress
cypress_restaurants = nearby_cypress_venues[nearby_cypress_venues['categories'].str.contains('Restaurant')]
cypress_restaurants.reset_index(drop = True, inplace = True)
print ('There are', len(cypress_restaurants),'restaurants in Cypress')
cypress_restaurants.head()

There are 18 restaurants in Cypress


,name,categories,lat,lng
0,Chick-fil-A,Fast Food Restaurant,29.969286,-95.693057
1,Alicia's Mexican Grille,Mexican Restaurant,29.974986,-95.701213
2,Chipotle Mexican Grill,Mexican Restaurant,29.972797,-95.691474
3,Green Garden,Chinese Restaurant,29.972137,-95.689471
4,Kolache Factory,Fast Food Restaurant,29.967291,-95.691580


In [17]:
#Finding other venues people might eat at after visiting in Cypress
cypress_stores = nearby_cypress_venues[nearby_cypress_venues['categories'].str.contains('Store')]
cypress_stores.reset_index(drop = True, inplace = True)
print ('There are', len(cypress_stores),'stores in Cypress')

cypress_shops = nearby_cypress_venues[nearby_cypress_venues['categories'].str.contains('Shop')]
cypress_shops.reset_index(drop = True, inplace = True)
print ('There are', len(cypress_shops),'shops and malls in Cypress')

cypress_beauty = nearby_cypress_venues[nearby_cypress_venues['categories'].str.contains('Beauty')]
cypress_beauty.reset_index(drop = True, inplace = True)
print ('There are', len(cypress_beauty),'beauty services in Cypress')

cypress_theater = nearby_cypress_venues[nearby_cypress_venues['categories'].str.contains('Theater')]
cypress_theater.reset_index(drop = True, inplace = True)
print ('There are', len(cypress_theater),'theaters in Cypress')

cypress_hotel = nearby_cypress_venues[nearby_cypress_venues['categories'].str.contains('Hotel')]
cypress_hotel.reset_index(drop = True, inplace = True)
print ('There are', len(cypress_hotel),'hotels in Cypress')

There are 14 stores in Cypress
There are 16 shops and malls in Cypress
There are 0 beauty services in Cypress
There are 1 theaters in Cypress
There are 0 hotels in Cypress


In [18]:
cypress_frames = [cypress_stores, cypress_shops, cypress_beauty, cypress_theater, cypress_hotel]

cypress_other_venues = pd.concat(cypress_frames)
cypress_other_venues.reset_index(drop = True, inplace = True)

print ('Considering stores, shops, beauty services, theaters, and hotels, there are', len(cypress_other_venues),'other venues in Cypress people might visit before eating')
cypress_other_venues.head()

Considering stores, shops, beauty services, theaters, and hotels, there are 31 other venues in Cypress people might visit before eating


,name,categories,lat,lng
0,Lowe's Home Improvement,Hardware Store,29.971483,-95.698571
1,T.J. Maxx,Department Store,29.968723,-95.695117
2,Best Buy,Electronics Store,29.965298,-95.690912
3,"Spec's Wines, Spirits & Finer Foods",Liquor Store,29.975445,-95.700679
4,Bed Bath & Beyond,Furniture / Home Store,29.968103,-95.694105


In [19]:
#Mapping all venues in Cypress
map_cypress = folium.Map(location=[cypress_latitude, cypress_longitude], zoom_start=13)

#Mapping restaurants in Cypress
for name, lat, lng in zip(cypress_restaurants['name'], cypress_restaurants['lat'], cypress_restaurants['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cypress)
    
#Mapping stores and shops in Cypress
for name, lat, lng in zip(cypress_stores['name'], cypress_stores['lat'], cypress_stores['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cypress)
for name, lat, lng in zip(cypress_shops['name'], cypress_shops['lat'], cypress_shops['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cypress)
    
#Mapping beauty services in Cypress
for name, lat, lng in zip(cypress_beauty['name'], cypress_beauty['lat'], cypress_beauty['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cypress)
    
#Mapping theaters in Cypress
for name, lat, lng in zip(cypress_theater['name'], cypress_theater['lat'], cypress_theater['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cypress)
    
#Mapping hotels in Cypress
for name, lat, lng in zip(cypress_hotel['name'], cypress_hotel['lat'], cypress_hotel['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cypress)
    
map_cypress

# Jersey Village

In [20]:
jv_limit = 100
jv_radius = 2710

#Defining URL's for Foursquare API Requests
jv_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, jv_latitude, jv_longitude, VERSION, jv_radius, jv_limit)

#Examining results
jv_results = requests.get(jv_url).json()


jv_venues = jv_results['response']['groups'][0]['items']
nearby_jv_venues = json_normalize(jv_venues)
filtered_jv_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_jv_venues =nearby_jv_venues.loc[:, filtered_jv_columns]
#     filter the category for each row
nearby_jv_venues['venue.categories'] = nearby_jv_venues.apply(get_category_type, axis=1)
#     clean columns
nearby_jv_venues.columns = [col.split(".")[-1] for col in nearby_jv_venues.columns]

In [21]:
#Finding Restaurants in Jersey Village
jv_restaurants = nearby_jv_venues[nearby_jv_venues['categories'].str.contains('Restaurant')]
jv_restaurants.reset_index(drop = True, inplace = True)
print ('There are', len(jv_restaurants),'restaurants in Jersey Village')
jv_restaurants.head()

There are 20 restaurants in Jersey Village


,name,categories,lat,lng
0,Adriatic Cafe & Italian Grill,Italian Restaurant,29.889968,-95.582447
1,Chick-fil-A,Fast Food Restaurant,29.904703,-95.552653
2,Cafe India,Indian Restaurant,29.901677,-95.552913
3,Mosa Asian Bistro,Asian Restaurant,29.902006,-95.552392
4,Chick-fil-A,Fast Food Restaurant,29.890599,-95.583521


In [22]:
#Finding other venues people might eat at after visiting in Jersey Village
jv_stores = nearby_jv_venues[nearby_jv_venues['categories'].str.contains('Store')]
jv_stores.reset_index(drop = True, inplace = True)
print ('There are', len(jv_stores),'stores in Jersey Village')

jv_shops = nearby_jv_venues[nearby_jv_venues['categories'].str.contains('Shop')]
jv_shops.reset_index(drop = True, inplace = True)
print ('There are', len(jv_shops),'shops and malls in Jersey Village')

jv_beauty = nearby_jv_venues[nearby_jv_venues['categories'].str.contains('Beauty')]
jv_beauty.reset_index(drop = True, inplace = True)
print ('There are', len(jv_beauty),'beauty services in Jersey Village')

jv_theater = nearby_jv_venues[nearby_jv_venues['categories'].str.contains('Theater')]
jv_theater.reset_index(drop = True, inplace = True)
print ('There are', len(jv_theater),'theaters in Jersey Village')

jv_hotel = nearby_jv_venues[nearby_jv_venues['categories'].str.contains('Hotel')]
jv_hotel.reset_index(drop = True, inplace = True)
print ('There are', len(jv_hotel),'hotels in Jersey Village')

There are 11 stores in Jersey Village
There are 11 shops and malls in Jersey Village
There are 0 beauty services in Jersey Village
There are 0 theaters in Jersey Village
There are 4 hotels in Jersey Village


In [23]:
jv_frames = [jv_stores, jv_shops, jv_beauty, jv_theater, jv_hotel]

jv_other_venues = pd.concat(jv_frames)
jv_other_venues.reset_index(drop = True, inplace = True)

print ('Considering stores, shops, beauty services, theaters, and hotels, there are', len(jv_other_venues),'other venues in Jersey Village people might visit before eating')
jv_other_venues.head()

Considering stores, shops, beauty services, theaters, and hotels, there are 26 other venues in Jersey Village people might visit before eating


,name,categories,lat,lng
0,"Spec's Wines, Spirits & Finer Foods",Liquor Store,29.890511,-95.582412
1,Dollar Tree,Discount Store,29.890791,-95.582664
2,Dollar Tree,Discount Store,29.902020,-95.553709
3,Redbox,Video Store,29.900818,-95.549858
4,Redbox,Video Store,29.903301,-95.553189


In [24]:
#Mapping all venues in Jersey Village
map_jv = folium.Map(location=[jv_latitude, jv_longitude], zoom_start=13)

#Mapping restaurants in Jersey Village
for name, lat, lng in zip(jv_restaurants['name'], jv_restaurants['lat'], jv_restaurants['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jv)
    
#Mapping stores and shops in Jersey Village
for name, lat, lng in zip(jv_stores['name'], jv_stores['lat'], jv_stores['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jv)
for name, lat, lng in zip(jv_shops['name'], jv_shops['lat'], jv_shops['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jv)
    
#Mapping beauty services in Jersey Village
for name, lat, lng in zip(jv_beauty['name'], jv_beauty['lat'], jv_beauty['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jv)
    
#Mapping theaters in Jersey Village
for name, lat, lng in zip(jv_theater['name'], jv_theater['lat'], jv_theater['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jv)
    
#Mapping hotels in Jersey Village
for name, lat, lng in zip(jv_hotel['name'], jv_hotel['lat'], jv_hotel['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jv)
    
map_jv

# Sugar Land

In [25]:
sl_limit = 100
sl_radius = 5900

#Defining URL's for Foursquare API Requests
sl_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, sl_latitude, sl_longitude, VERSION, sl_radius, sl_limit)

#Examining results
sl_results = requests.get(sl_url).json()

sl_venues = sl_results['response']['groups'][0]['items']
nearby_sl_venues = json_normalize(sl_venues)
filtered_sl_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_sl_venues =nearby_sl_venues.loc[:, filtered_sl_columns]
#     filter the category for each row
nearby_sl_venues['venue.categories'] = nearby_sl_venues.apply(get_category_type, axis=1)
#     clean columns
nearby_sl_venues.columns = [col.split(".")[-1] for col in nearby_sl_venues.columns]

In [26]:
#Finding Restaurants in Sugar Land
sl_restaurants = nearby_sl_venues[nearby_sl_venues['categories'].str.contains('Restaurant')]
sl_restaurants.reset_index(drop = True, inplace = True)
print ('There are', len(sl_restaurants),'restaurants in Sugar Land')
sl_restaurants.head()

There are 36 restaurants in Sugar Land


,name,categories,lat,lng
0,Fadis Mediterranean Grill,Mediterranean Restaurant,29.606914,-95.642116
1,Chuy's Sugar Land,Mexican Restaurant,29.610223,-95.648583
2,Pei Wei,Asian Restaurant,29.598284,-95.625977
3,Azuma on the Lake,Japanese Restaurant,29.601204,-95.621858
4,Japaneiro’s Sushi Bistro & Latin Grill,Sushi Restaurant,29.597434,-95.621554


In [27]:
#Finding other venues people might eat at after visiting in Sugar Land
sl_stores = nearby_sl_venues[nearby_sl_venues['categories'].str.contains('Store')]
sl_stores.reset_index(drop = True, inplace = True)
print ('There are', len(sl_stores),'stores in Sugar Land')

sl_shops = nearby_sl_venues[nearby_sl_venues['categories'].str.contains('Shop')]
sl_shops.reset_index(drop = True, inplace = True)
print ('There are', len(sl_shops),'shops and malls in Sugar Land')

sl_beauty = nearby_sl_venues[nearby_sl_venues['categories'].str.contains('Beauty')]
sl_beauty.reset_index(drop = True, inplace = True)
print ('There are', len(sl_beauty),'beauty services in Sugar Land')

sl_theater = nearby_sl_venues[nearby_sl_venues['categories'].str.contains('Theater')]
sl_theater.reset_index(drop = True, inplace = True)
print ('There are', len(sl_theater),'theaters in Sugar Land')

sl_hotel = nearby_sl_venues[nearby_sl_venues['categories'].str.contains('Hotel')]
sl_hotel.reset_index(drop = True, inplace = True)
print ('There are', len(sl_hotel),'hotels in Sugar Land')

There are 19 stores in Sugar Land
There are 10 shops and malls in Sugar Land
There are 2 beauty services in Sugar Land
There are 0 theaters in Sugar Land
There are 2 hotels in Sugar Land


In [28]:
sl_frames = [sl_stores, sl_shops, sl_beauty, sl_theater, sl_hotel]

sl_other_venues = pd.concat(sl_frames)
sl_other_venues.reset_index(drop = True, inplace = True)

print ('Considering stores, shops, beauty services, theaters, and hotels, there are', len(sl_other_venues),'other venues in Sugar Land people might visit before eating')
sl_other_venues.head()

Considering stores, shops, beauty services, theaters, and hotels, there are 33 other venues in Sugar Land people might visit before eating


,name,categories,lat,lng
0,H-E-B,Grocery Store,29.607822,-95.645884
1,Sam's Club,Warehouse Store,29.612143,-95.642673
2,Whole Foods Market,Grocery Store,29.599883,-95.623388
3,Macy's,Department Store,29.593289,-95.625395
4,Total Wine & More,Liquor Store,29.593115,-95.636305


In [29]:
#Mapping all venues in Sugar Land
map_sl = folium.Map(location=[sl_latitude, sl_longitude], zoom_start=13)

#Mapping restaurants in Sugar Land
for name, lat, lng in zip(sl_restaurants['name'], sl_restaurants['lat'], sl_restaurants['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='orange',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sl)
    
#Mapping stores and shops in Sugar Land
for name, lat, lng in zip(sl_stores['name'], sl_stores['lat'], sl_stores['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sl)
for name, lat, lng in zip(sl_shops['name'], sl_shops['lat'], sl_shops['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sl)
    
#Mapping beauty services in Sugar Land
for name, lat, lng in zip(sl_beauty['name'], sl_beauty['lat'], sl_beauty['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='pink',
        fill=True,
        fill_color='pink',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sl)
    
#Mapping theaters in Sugar Land
for name, lat, lng in zip(sl_theater['name'], sl_theater['lat'], sl_theater['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sl)
    
#Mapping hotels in Sugar Land
for name, lat, lng in zip(sl_hotel['name'], sl_hotel['lat'], sl_hotel['lng']):
    label = '{}'.format(name),
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sl)
    
map_sl

# Summarizing Data

In [30]:
summarized_data = {'Area': ['Memorial City', 'Katy', 'Cypress', 'Jersey Village', 'Sugar Land'],
        'Number of Restaurants in Suburban Area': [len(memorial_restaurants), len(katy_restaurants), len(cypress_restaurants), len(jv_restaurants), len(sl_restaurants)],
        'Number of Other Venues in Suburban Areas': [len(memorial_other_venues), len(katy_other_venues), len(cypress_other_venues), len(jv_other_venues), len(sl_other_venues)]}

summarized_df = pd.DataFrame(summarized_data, columns = ['Area', 'Number of Restaurants in Suburban Area','Number of Other Venues in Suburban Areas'])
summarized_df

,Area,Number of Restaurants in Suburban Area,Number of Other Venues in Suburban Areas
0,Memorial City,38,36
1,Katy,27,43
2,Cypress,18,31
3,Jersey Village,20,26
4,Sugar Land,36,33


## **3. Methodology**

While there are many more suburban areas in the city of Houston, the five discussed in this report were chosen so that they were far enough apart, and the same venues wouldn’t pop up for multiple suburban areas. Also, as seen in the corresponding notebook with the code used in this project, the radius of each area was selected based on the relative size of each suburban area. This was done to ensure that all the venue and location data was included and not left out due to not being in the set radius, but still within the area.

Because I wanted to view how many other restaurants were in the area, venues that had the word “restaurant” in their category label were transferred to a restaurants dataframe. Other categories of interest in this project were venues that people might visit either just before or right after eating. For simplicity, the venues analyzed were stores/shops, beauty salons, movie theaters, and hotels.

For each of the previously shown maps, the colors orange, blue, pink, red, and green are representative of restaurants, stores/shops, beauty services, movie theaters, and hotels, respectively. To determine the best location to open a restaurant, it would be important to not only have a list of these venues, but also maps of them. It would be unwise to open a restaurant in close proximity to several other restaurants, even if it was near to a movie theater or hotel, etc. 

## **4. Results and Discussion**

Quantifying the numbers of restaurants and other venues in these suburban areas helps determine which area is ideal for a new business. The number of restaurants in the area will determine how much competition is in the area. Opening a restaurant near stores or hotels would be wise because potential customers won’t have to go far in order to purchase a meal.

The area with the fewest number of restaurants is Cypress, while the area with the most stores/shops, hotels, theaters (other venues) is Katy. Upon analysis of the maps shown in the data section, although Katy has the most locations in our “other venues” category, it would not be wise to open a restaurant near these. This is due to the fact that the location of all the shops/stores, hotels, theaters, and beauty services are very saturated with restaurants. Opening a restaurant in Katy solely based on the fact that it has several other venues would not be the best decision because these locations are already crowded and clustered with several other restaurants.

The Jersey Village map shows that this area could be a potential place to open a successful business, mainly because there aren’t many types of venues that we’re analyzing in the area to begin with. However, the lack of data does not always deliver the best results, it would be analogous to a shot in the dark. Out of the five areas, Sugar Land has the second highest number of restaurants in this area. While Sugar Land does have several venues in our “other venues” category, these are also surrounded by many restaurants in the area, making it an area of high competition for a new eatery. Memorial City and Sugar Land are similar in the fact that they both have a large amount of restaurants (Memorial City has the largest number of restaurants in the area) and the restaurants surround many of the hotels, theaters, stores/shops, etc. This would mean Memorial City is also not the best fit to ensure a successful and thriving business.

Analysis of the Cypress map, not only shows the fewest number of restaurants in all of the suburban areas of interest, but many of the stores/shops, hotels, and theaters are distanced from other restaurants in Cypress. The location data acquiring using Foursquare API clearly indicates that Cypress would be the best out of the five suburban areas to open up a restaurant.

Upon completion of this project, the analysis has been to discovered to have room for improvement in various areas. When acquiring the location data, the number of restaurants was chosen based on if the category column explicitly contained the word “restaurant”. However, there were discovered to be venues that were sports bars, steakhouses, or burger joints. While these venues are also places where people can eat, they did not show up in our filtered column because they did not contain the phrase “restaurant”. Factoring for these possibilities in the future could alter the location data and potentially change the results.

Another way to improve this model is to also obtain tips from the Foursquare API data. It would be a good idea to open a restaurant around venues that had good tips and very positive ratings. These venues are very popular and attract a lot of potential customers to these areas.

## **6. Conclusion**

In conclusion, it appears that Cypress would be the best choice to open a restaurant. This choice is driven by location data of various suburban areas in West Houston. Categories of venues in these areas are extracted by key words and analyzed in order to determine which area had higher chance of success. Not only were the numbers of these venues considered, but also the location of them with respect to each other. This analysis could be improved by including other types of eateries other than explicitly “restaurants”. However, this analysis of location data provides very valuable insight to potential stakeholders, investors, and business owners.